### Notes:

This version of the notebook is used for running the most recent version YOLO v5 which has feature maps to show which featuers are the most important. It can be used for re-training the model as needed.

In [1]:
import os
os.chdir("D:\\Datasets\\SIIM_RSNA_Covid_Datasets")
os.getcwd()

'D:\\Datasets\\SIIM_RSNA_Covid_Datasets'

In [2]:
# Set up OS and run to get YOLO v5 in the tmp folder

!mkdir tmp
%cd tmp
# %cd ../

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp
D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp\yolov5


Cloning into 'yolov5'...


In [2]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Install W&B 
# !pip install -q --upgrade wandb
# Login 

import wandb
wandb.login()

# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Setup complete. Using torch 1.8.1 (NVIDIA GeForce GTX 1080 Ti)


wandb: Currently logged in as: ogk01 (use `wandb login --relogin` to force relogin)


### Hyperparameters

In [4]:
TRAIN_PATH = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\train_jpeg_512\\'
train_image_level_filepath = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\train_image_level.csv'
meta_filepath = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\meta_256.csv'
yolo_yaml_filepath = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\yolov5\\data\\data.yaml'
IMG_SIZE = 512
BATCH_SIZE = 64
EPOCHS = 50

### Prepare Dataset

This is the most important section when it comes to training an object detector with YOLOv5. The directory structure, bounding box format, etc must be in the correct order. This section builds every piece needed to train a YOLOv5 model.

Using [xhlulu's](https://www.kaggle.com/xhlulu) resized dataset. The uploaded 256x256 Kaggle dataset is [here](https://www.kaggle.com/xhlulu/siim-covid19-resized-to-256px-jpg). Find other image resolutions [here](https://www.kaggle.com/c/siim-covid19-detection/discussion/239918).

* Create train-validation split. <br>
* Create required `/dataset` folder structure and more the images to that folder. <br>
* Create `data.yaml` file needed to train the model. <br>
* Create bounding box coordinates in the required YOLO format. 

In [5]:
# Everything is done from /kaggle directory.
#%cd ../

# Load image level csv file
df = pd.read_csv(train_image_level_filepath)

# Modify values in the id column
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)

# Add absolute path
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.jpg', axis=1)

# Get image level labels
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)

df.head(5)

# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.
meta_df = pd.read_csv(meta_filepath)
train_meta_df = meta_df.loc[meta_df.split == 'train']
train_meta_df = train_meta_df.drop('split', axis=1)
train_meta_df.columns = ['id', 'dim0', 'dim1']
train_meta_df.head(2)

# Merge both the dataframes
df = df.merge(train_meta_df, on='id',how="left")
df.head(2)

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,D:\Datasets\SIIM_RSNA_Covid_Datasets\train_jpe...,opacity,3488,4256
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,D:\Datasets\SIIM_RSNA_Covid_Datasets\train_jpe...,none,2320,2832


## 🍘 Train-validation split

In [6]:
# Create train and validation split.
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df.image_level.values)

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)

print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')

Size of dataset: 6334, training images: 5067. validation images: 1267


D:\Anaconda\envs\Pytorch\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\Anaconda\envs\Pytorch\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 🍚 Prepare Required Folder Structure

The required folder structure for the dataset directory is: 

```
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Note that I have named the directory `covid`.

In [7]:
os.chdir("D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\")
os.makedirs('tmp/covid/images/train', exist_ok=True)
os.makedirs('tmp/covid/images/valid', exist_ok=True)

os.makedirs('tmp/covid/labels/train', exist_ok=True)
os.makedirs('tmp/covid/labels/valid', exist_ok=True)

#!ls tmp/covid/images

# Move the images to relevant split folder.
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\covid\\images\\train\\{row.id}.jpg')
    else:
        copyfile(row.path, f'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\covid\\images\\valid\\{row.id}.jpg')

100%|█████████████████████████████████████████████████████████████████████████████| 6334/6334 [00:07<00:00, 828.51it/s]


#### Copy the images into the tmp folder created above

## 🍜 Create `.YAML` file

The `data.yaml`, is the dataset configuration file that defines 

1. an "optional" download command/URL for auto-downloading, 
2. a path to a directory of training images (or path to a *.txt file with a list of training images), 
3. a path to a directory of validation images (or path to a *.txt file with a list of validation images), 
4. the number of classes, 
5. a list of class names.

> 📍 Important: In this competition, each image can either belong to `opacity` or `none` image-level labels. That's why I have  used the number of classes, `nc` to be 2. YOLOv5 automatically handles the images without any bounding box coordinates. 

> 📍 Note: The `data.yaml` is created in the `yolov5/data` directory as required. 

In [8]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '../covid/images/train',
    val = '../covid/images/valid',
    nc = 2,
    names = ['none', 'opacity']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open(yolo_yaml_filepath, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

## 🍮 Prepare Bounding Box Coordinated for YOLOv5

For every image with **bounding box(es)** a `.txt` file with the same name as the image will be created in the format shown below:

* One row per object. <br>
* Each row is class `x_center y_center width height format`. <br>
* Box coordinates must be in normalized xywh format (from 0 - 1). We can normalize by the boxes in pixels by dividing `x_center` and `width` by image width, and `y_center` and `height` by image height. <br>
* Class numbers are zero-indexed (start from 0). <br>

> 📍 Note: We don't have to remove the images without bounding boxes from the training or validation sets. 

In [9]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

# Prepare the txt files for bounding box
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.id
    # Get split
    split = row.split
    # Get image-level label
    label = row.image_level
    
    if row.split=='train':
        file_name = f'tmp/covid/labels/train/{row.id}.txt'
    else:
        file_name = f'tmp/covid/labels/valid/{row.id}.txt'
        
    
    if label=='opacity':
        # Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row, bboxes)
        # Format for YOLOv5
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

100%|████████████████████████████████████████████████████████████████████████████| 6334/6334 [00:03<00:00, 1739.63it/s]


# 🚅 Train with W&B



#%cd tmp/yolov5/
```
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--weights yolov5s.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name
```

In [5]:
os.chdir("D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\yolov5")
os.getcwd()

'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\yolov5'

In [6]:
# Original Train
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project SIIM_COVID_512_Runs

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=64, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=SIIM_COVID_512_Runs, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 

Plotting labels... 

autoanchor: Analyzing anchors... anchors/target = 4.76, Best Possible Recall (BPR) = 1.0000
                 all       1267       1582      0.145      0.285     0.0872     0.0169
Saving model artifact on epoch  1
                 all       1267       1582      0.299      0.336      0.232     0.0566
Saving model artifact on epoch  2
                 all       1267

YOLOv5  v5.0-350-gf409d8e torch 1.8.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264.0MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir SIIM_COVID_512_Runs', view at http://localhost:6006/
wandb: Currently logged in as: ogk01 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.30
wandb: Syncing run lively-salad-2
wandb:  View project at https://wandb.ai/ogk01/SIIM_COVID_512_Runs
wandb:  View run at https://wandb.ai/ogk01/SIIM_COVID_512_Runs/runs/1l6hrlnf
wandb: Run da

Saving model artifact on epoch  14
                 all       1267       1582      0.471      0.419      0.364     0.0955
Saving model artifact on epoch  15
                 all       1267       1582      0.321      0.292      0.214     0.0524
Saving model artifact on epoch  16
                 all       1267       1582      0.367       0.39      0.293     0.0686
Saving model artifact on epoch  17
                 all       1267       1582      0.427      0.414      0.352     0.0929
Saving model artifact on epoch  18
                 all       1267       1582      0.532      0.434      0.433      0.126
Saving model artifact on epoch  19
                 all       1267       1582      0.495      0.451      0.395      0.106
Saving model artifact on epoch  20
                 all       1267       1582      0.508      0.399      0.387      0.108
Saving model artifact on epoch  21
                 all       1267       1582      0.516      0.444      0.407      0.111
Saving model artifact on


  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1    656896  models.common.SPP                       [512, 512, [5, 9, 13]]        
  9                -1  1   1182720  models.common.C3                        [512, 512, 1, False]          
 10                -1  1    131584  models.common.Conv                      [512, 256, 1, 1]              
 11                -1  1         0  

      0/49     7.97G   0.09374   0.02412   0.01589       158       512:  80%|########  | 64/80 [01:06<00:15,  1.02it/s]
      0/49     7.97G   0.09348    0.0241   0.01573       120       512:  80%|########  | 64/80 [01:07<00:15,  1.02it/s]
      0/49     7.97G   0.09348    0.0241   0.01573       120       512:  81%|########1 | 65/80 [01:07<00:14,  1.01it/s]
      0/49     7.97G   0.09325   0.02405   0.01557       106       512:  81%|########1 | 65/80 [01:08<00:14,  1.01it/s]
      0/49     7.97G   0.09325   0.02405   0.01557       106       512:  82%|########2 | 66/80 [01:08<00:13,  1.01it/s]
      0/49     7.97G     0.093    0.0241   0.01541       152       512:  82%|########2 | 66/80 [01:09<00:13,  1.01it/s]
      0/49     7.97G     0.093    0.0241   0.01541       152       512:  84%|########3 | 67/80 [01:09<00:12,  1.01it/s]
      0/49     7.97G   0.09276   0.02413   0.01526       140       512:  84%|########3 | 67/80 [01:10<00:12,  1.01it/s]
      0/49     7.97G   0.09276   0.02413

     17/49     9.19G   0.05131   0.01975  0.000132       153       512:  57%|#####7    | 46/80 [00:28<00:20,  1.63it/s]
     17/49     9.19G   0.05128   0.01977 0.0001319       147       512:  57%|#####7    | 46/80 [00:29<00:20,  1.63it/s]
     17/49     9.19G   0.05128   0.01977 0.0001319       147       512:  59%|#####8    | 47/80 [00:29<00:20,  1.63it/s]
     17/49     9.19G   0.05128   0.01973 0.0001317       122       512:  59%|#####8    | 47/80 [00:29<00:20,  1.63it/s]
     17/49     9.19G   0.05128   0.01973 0.0001317       122       512:  60%|######    | 48/80 [00:29<00:19,  1.62it/s]
     17/49     9.19G   0.05124   0.01976 0.0001316       153       512:  60%|######    | 48/80 [00:30<00:19,  1.62it/s]
     17/49     9.19G   0.05124   0.01976 0.0001316       153       512:  61%|######1   | 49/80 [00:30<00:19,  1.63it/s]
     17/49     9.19G   0.05123   0.01974 0.0001319       127       512:  61%|######1   | 49/80 [00:31<00:19,  1.63it/s]
     17/49     9.19G   0.05123   0.01974

     22/49     9.19G    0.0494   0.01952 0.0001105       145       512:  14%|#3        | 11/80 [00:06<00:36,  1.90it/s]
     22/49     9.19G   0.04957   0.01925 0.0001091       119       512:  14%|#3        | 11/80 [00:06<00:36,  1.90it/s]
     22/49     9.19G   0.04957   0.01925 0.0001091       119       512:  15%|#5        | 12/80 [00:06<00:35,  1.91it/s]
     22/49     9.19G   0.04947   0.01917 0.0001086       130       512:  15%|#5        | 12/80 [00:07<00:35,  1.91it/s]
     22/49     9.19G   0.04947   0.01917 0.0001086       130       512:  16%|#6        | 13/80 [00:07<00:35,  1.89it/s]
     22/49     9.19G   0.04948   0.01927  0.000108       138       512:  16%|#6        | 13/80 [00:07<00:35,  1.89it/s]
     22/49     9.19G   0.04948   0.01927  0.000108       138       512:  18%|#7        | 14/80 [00:07<00:36,  1.82it/s]
     22/49     9.19G   0.04923   0.01938 0.0001081       144       512:  18%|#7        | 14/80 [00:08<00:36,  1.82it/s]
     22/49     9.19G   0.04923   0.01938

     33/49     9.19G   0.04838   0.01907 5.973e-05       133       512:   1%|1         | 1/80 [00:01<00:40,  1.96it/s]
     33/49     9.19G   0.04838   0.01907 5.973e-05       133       512:   2%|2         | 2/80 [00:01<00:39,  1.96it/s]
     33/49     9.19G   0.04785   0.01877 6.264e-05       121       512:   2%|2         | 2/80 [00:01<00:39,  1.96it/s]
     33/49     9.19G   0.04785   0.01877 6.264e-05       121       512:   4%|3         | 3/80 [00:01<00:39,  1.95it/s]
     33/49     9.19G   0.04807    0.0193 6.389e-05       146       512:   4%|3         | 3/80 [00:02<00:39,  1.95it/s]
     33/49     9.19G   0.04807    0.0193 6.389e-05       146       512:   5%|5         | 4/80 [00:02<00:39,  1.94it/s]
     33/49     9.19G   0.04836   0.01976  6.51e-05       153       512:   5%|5         | 4/80 [00:02<00:39,  1.94it/s]
     33/49     9.19G   0.04836   0.01976  6.51e-05       153       512:   6%|6         | 5/80 [00:02<00:38,  1.93it/s]
     33/49     9.19G   0.04823   0.01947  6.57e-

     33/49     9.19G   0.04738   0.01944 6.654e-05       124       512:  96%|#########6| 77/80 [00:40<00:01,  1.84it/s]
     33/49     9.19G   0.04737   0.01942 6.649e-05       137       512:  96%|#########6| 77/80 [00:41<00:01,  1.84it/s]
     33/49     9.19G   0.04737   0.01942 6.649e-05       137       512:  98%|#########7| 78/80 [00:41<00:01,  1.85it/s]
     33/49     9.19G    0.0474   0.01943 6.648e-05       145       512:  98%|#########7| 78/80 [00:41<00:01,  1.85it/s]
     33/49     9.19G    0.0474   0.01943 6.648e-05       145       512:  99%|#########8| 79/80 [00:41<00:00,  1.87it/s]
     33/49     9.19G    0.0474   0.01947 6.661e-05        26       512:  99%|#########8| 79/80 [00:41<00:00,  1.87it/s]
     33/49     9.19G    0.0474   0.01947 6.661e-05        26       512: 100%|##########| 80/80 [00:41<00:00,  2.40it/s]
     33/49     9.19G    0.0474   0.01947 6.661e-05        26       512: 100%|##########| 80/80 [00:41<00:00,  1.91it/s]

               Class     Images     Lab

     34/49     9.19G   0.04723   0.01916 6.614e-05       128       512:  80%|########  | 64/80 [00:34<00:08,  1.88it/s]
     34/49     9.19G   0.04723   0.01916 6.614e-05       128       512:  81%|########1 | 65/80 [00:34<00:07,  1.89it/s]
     34/49     9.19G   0.04721   0.01917 6.619e-05       153       512:  81%|########1 | 65/80 [00:34<00:07,  1.89it/s]
     34/49     9.19G   0.04721   0.01917 6.619e-05       153       512:  82%|########2 | 66/80 [00:34<00:07,  1.91it/s]
     34/49     9.19G   0.04718   0.01915 6.611e-05       122       512:  82%|########2 | 66/80 [00:35<00:07,  1.91it/s]
     34/49     9.19G   0.04718   0.01915 6.611e-05       122       512:  84%|########3 | 67/80 [00:35<00:06,  1.92it/s]
     34/49     9.19G   0.04721   0.01917  6.61e-05       148       512:  84%|########3 | 67/80 [00:35<00:06,  1.92it/s]
     34/49     9.19G   0.04721   0.01917  6.61e-05       148       512:  85%|########5 | 68/80 [00:35<00:06,  1.92it/s]
     34/49     9.19G   0.04719   0.01919

### Hyperparameter Tuning (Multiple Runs)

In [ ]:
# Manually Tune Hyperparameters to test

!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --hyp data/hyps/hyp.scratch_1.yaml \
                 --project SIIM_COVID_512_Runs

In [14]:
# Evolve Hyperparameters - This can be time consuming, so be careful

!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs 20 \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --evolve 20 \
                 --project SIIM_COVID_512_Runs

^C


## Model Saved Automatically as Artifact

Since it's a kernel based competition, you can easily download the best model from the W&B Artifacts UI and upload as a Kaggle dataset that you can load in your inference kernel (internel disabled).

### [Path to saved model $\rightarrow$](https://wandb.ai/ayush-thakur/kaggle-siim-covid/artifacts/model/run_jbt74n7q_model/4c3ca5752dba99bd227e)

![img](https://i.imgur.com/KhRLQvR.png)

> 📍 Download the model with the `best` alias tagged to it. 

# Inference

You will probably use a `Submission.ipynb` kernel to run all the predictions. After training a YOLOv5 based object detector -> head to the artifacts page and download the best model -> upload the model as a Kaggle dataset -> Use it with the submission folder. 

> 📍 Note that you might have to clone the YOLOv5 repository in a Kaggle dataset as well. 

In this section, I will show you how you can do the inference and modify the predicted bounding box coordinates.

In [7]:
# absolute path
TEST_PATH = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\test_jpeg_512'

Since I am training the model in this kernel itself, I will not be using the method that I have described above. The best model is saved in the directory `project_name/exp*/weights/best.pt`. In `exp*`, * can be 1, 2, etc. 

In [11]:
MODEL_PATH = 'SIIM_COVID_512_Runs/exp2/weights/best.pt'

```
--weights {MODEL_PATH} \ # path to the best model.
--source {TEST_PATH} \ # absolute path to the test images.
--img {IMG_SIZE} \ # Size of image
--conf 0.281 \ # Confidence threshold (default is 0.25)
--iou-thres 0.5 \ # IOU threshold (default is 0.45)
--max-det 3 \ # Number of detections per image (default is 1000) 
--save-txt \ # Save predicted bounding box coordinates as txt files
--save-conf # Save the confidence of prediction for each bounding box
```

In [12]:
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf \
                  --visualize

detect: weights=['SIIM_COVID_512_Runs/exp2/weights/best.pt'], source=D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg_512, imgsz=512, conf_thres=0.281, iou_thres=0.5, max_det=3, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=True, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
image 1/180 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg_512\0026720152f5.jpg: Saving runs\detect\exp2\0026720152f5\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\0026720152f5\stage1_Conv_features.png... (32/64)
Saving runs\detect\exp2\0026720152f5\stage2_C3_features.png... (32/64)
Saving runs\detect\exp2\0026720152f5\stage3_Conv_features.png... (32/128)
Saving runs\detect\exp2\0026720152f5\stage4_C3_features.png... (32/128)
Saving runs\detect\exp2\0026720152f5\stage5_Conv_features.png... (32/256)
Saving runs\detect\exp2\002

YOLOv5  v5.0-350-gf409d8e torch 1.8.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264.0MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPs
D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp\yolov5\utils\plots.py:424: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(math.ceil(n / 8), 8, tight_layout=True)  # 8 rows x n/8 cols



Saving runs\detect\exp2\0026720152f5\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\0026720152f5\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\0026720152f5\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\0026720152f5\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\0026720152f5\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\0026720152f5\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\0026720152f5\stage22_Concat_features.png... (32/512)
Saving runs\detect\exp2\0026720152f5\stage23_C3_features.png... (32/512)
512x512 Done. (20.110s)
image 2/180 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg_512\00fc8fc35dc1.jpg: Saving runs\detect\exp2\00fc8fc35dc1\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\00fc8fc35dc1\stage1_Conv_features.png... (32/64)
Saving runs\detect\exp2\00fc8fc35dc1\stage2_C3_features.png... (32/64)
Saving runs\detect\exp2\00fc8fc35dc1\stage3_Conv_features.png... (32/128)
Sa

In [13]:
from datetime import datetime
now = datetime.now() 
print("now =", now)

Saving runs\detect\exp2\01f948f8e544\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\01f948f8e544\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\01f948f8e544\stage22_Concat_features.png... (32/512)
Saving runs\detect\exp2\01f948f8e544\stage23_C3_features.png... (32/512)
512x512 Done. (20.507s)
image 5/180 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg_512\022146012034.jpg: Saving runs\detect\exp2\022146012034\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\022146012034\stage1_Conv_features.png... (32/64)
Saving runs\detect\exp2\022146012034\stage2_C3_features.png... (32/64)
Saving runs\detect\exp2\022146012034\stage3_Conv_features.png... (32/128)
Saving runs\detect\exp2\022146012034\stage4_C3_features.png... (32/128)
Saving runs\detect\exp2\022146012034\stage5_Conv_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage6_C3_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage7_Conv_features.png... (32/512)
Saving runs\d


Saving runs\detect\exp2\04720d7e4e42\stage9_C3_features.png... (32/512)
Saving runs\detect\exp2\04720d7e4e42\stage10_Conv_features.png... (32/256)
Saving runs\detect\exp2\04720d7e4e42\stage11_Upsample_features.png... (32/256)
Saving runs\detect\exp2\04720d7e4e42\stage12_Concat_features.png... (32/512)
Saving runs\detect\exp2\04720d7e4e42\stage13_C3_features.png... (32/256)
Saving runs\detect\exp2\04720d7e4e42\stage14_Conv_features.png... (32/128)
Saving runs\detect\exp2\04720d7e4e42\stage15_Upsample_features.png... (32/128)
Saving runs\detect\exp2\04720d7e4e42\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\04720d7e4e42\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\04720d7e4e42\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\04720d7e4e42\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\04720d7e4e42\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\04720d7e4e42\stage21_Conv_features.png... (32/256)
Saving runs\detect\

Saving runs\detect\exp2\05435da60872\stage15_Upsample_features.png... (32/128)
Saving runs\detect\exp2\05435da60872\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\05435da60872\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\05435da60872\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\05435da60872\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\05435da60872\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\05435da60872\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\05435da60872\stage22_Concat_features.png... (32/512)
Saving runs\detect\exp2\05435da60872\stage23_C3_features.png... (32/512)
512x512 1 opacity, Done. (18.801s)
image 22/180 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg_512\0562436fde50.jpg: Saving runs\detect\exp2\0562436fde50\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\0562436fde50\stage1_Conv_features.png... (32/64)
Saving runs\detect\exp2\0562436fde50\stage2_C3_features.p

Saving runs\detect\exp2\144c27a2554a\stage10_Conv_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage11_Upsample_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage12_Concat_features.png... (32/512)
Saving runs\detect\exp2\144c27a2554a\stage13_C3_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage14_Conv_features.png... (32/128)
Saving runs\detect\exp2\144c27a2554a\stage15_Upsample_features.png... (32/128)
Saving runs\detect\exp2\144c27a2554a\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\144c27a2554a\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\144c27a2554a\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\144c27a2554a\stage22_Concat_features.png... (32/512)
Saving runs\det

Saving runs\detect\exp2\18e6a4083920\stage14_Conv_features.png... (32/128)
Saving runs\detect\exp2\18e6a4083920\stage15_Upsample_features.png... (32/128)
Saving runs\detect\exp2\18e6a4083920\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\18e6a4083920\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\18e6a4083920\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\18e6a4083920\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\18e6a4083920\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\18e6a4083920\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\18e6a4083920\stage22_Concat_features.png... (32/512)
Saving runs\detect\exp2\18e6a4083920\stage23_C3_features.png... (32/512)
512x512 Done. (19.846s)
image 127/180 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg_512\198ffac65dac.jpg: Saving runs\detect\exp2\198ffac65dac\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\198ffac65dac\stage1_Conv_features.png... 